In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# === Parameters ===
MOM_WINDOW = 2        # lookback window (months) for momentum
VOL_WINDOW = 3        # lookback window (months) for volatility
COV_WINDOW = 36        # lookback window (months) for covariance estimation
TARGET_VOL = 0.01      # 1% annual volatility target

# === Load Returns Data ===
xlsx = pd.ExcelFile('./data/Data1.xlsx')
returns = xlsx.parse(sheet_name=xlsx.sheet_names[0], index_col=0, parse_dates=True)

# === Compute Factor Signals ===
momentum = -(1 + returns).rolling(window=MOM_WINDOW).apply(np.prod) - 1
vol = returns.rolling(window=VOL_WINDOW).std() * np.sqrt(12)

# === Cross-Sectional Z-Scores ===
z_mom = momentum.sub(momentum.mean(axis=1), axis=0).div(momentum.std(axis=1), axis=0)
z_vol = vol.sub(vol.mean(axis=1), axis=0).div(vol.std(axis=1), axis=0)

# === Raw Factor Weights (sum of abs weights = 1) ===
pf_mom = z_mom.div(z_mom.abs().sum(axis=1), axis=0)
pf_vol = z_vol.div(z_vol.abs().sum(axis=1), axis=0)

# === Scaling & Combination ===
dates = returns.index[COV_WINDOW:]
w_mom_1 = pd.DataFrame(index=dates, columns=returns.columns, dtype=float)
w_vol_1 = pd.DataFrame(index=dates, columns=returns.columns, dtype=float)
w_comb = pd.DataFrame(index=dates, columns=returns.columns, dtype=float)

for date in dates:
    if date in pf_mom.index and date in pf_vol.index:
        raw_m = pf_mom.loc[date]
        raw_v = pf_vol.loc[date]
        cov = returns.loc[:date].iloc[-COV_WINDOW:].cov()
        vol_m = np.sqrt(raw_m.values @ cov.values @ raw_m.values * 12)
        vol_v = np.sqrt(raw_v.values @ cov.values @ raw_v.values * 12)
        w_m1 = raw_m * (TARGET_VOL / vol_m)
        w_v1 = raw_v * (TARGET_VOL / vol_v)
        w_c = 0.5 * w_m1 + 0.5 * w_v1
        vol_c = np.sqrt(w_c.values @ cov.values @ w_c.values * 12)
        w_comb.loc[date] = w_c * (TARGET_VOL / vol_c)
        w_mom_1.loc[date] = w_m1
        w_vol_1.loc[date] = w_v1

# Calculate strategy returns
ret_mom = (returns * w_mom_1.shift()).sum(axis=1).dropna()
ret_volf = (returns * w_vol_1.shift()).sum(axis=1).dropna()
ret_port = (returns * w_comb.shift()).sum(axis=1).dropna()

# Cumulative growth
cum = pd.DataFrame({
    'Momentum Factor': (1+ret_mom).cumprod(),
    'Volatility Factor': (1+ret_volf).cumprod(),
    'Combined Portfolio': (1+ret_port).cumprod()
})

# Performance metrics
ann_ret = cum.pct_change().mean() * 12
ann_vol = cum.pct_change().std() * np.sqrt(12)
# Information ratio vs momentum
ir = (ret_port - ret_mom).mean() / (ret_port - ret_mom).std() * np.sqrt(12)
# Drawdowns
wealth = cum['Combined Portfolio']
dd = wealth / wealth.cummax() - 1
avg_dd = dd.mean()
max_dd = dd.min()
# Turnover
turn = w_comb.diff().abs().sum(axis=1).mean()

metrics = pd.Series({
    'Annualized Return': ((1+ret_port).prod()**(12/len(ret_port)) - 1),
    'Annualized Volatility': ret_port.std() * np.sqrt(12),
    'Information Ratio (vs Momentum)': ir,
    'Average Drawdown': avg_dd,
    'Maximum Drawdown': max_dd,
    'Average Turnover': turn
})

# Final period covariance, volatilities, correlations
last_date = dates[-1]
cov_final = returns.loc[:last_date].iloc[-COV_WINDOW:].cov() * 12
vol_final = np.sqrt(np.diag(cov_final))
corr_final = cov_final.div(vol_final, axis=0).div(vol_final, axis=1)

# Save to Excel
# 
with pd.ExcelWriter('./data/backtest_report.xlsx') as writer:
    cum.to_excel(writer, sheet_name='Cumulative Growth')
    metrics.to_frame('Value').to_excel(writer, sheet_name='Performance Metrics')
    pf_mom.loc[dates].to_excel(writer, sheet_name='Raw Momentum Holdings')
    pf_vol.loc[dates].to_excel(writer, sheet_name='Raw Vol Holdings')
    w_mom_1.to_excel(writer, sheet_name='Momentum 1% Vol Holdings')
    w_vol_1.to_excel(writer, sheet_name='Vol Factor 1% Vol Holdings')
    w_comb.to_excel(writer, sheet_name='Final 1% Vol Holdings')
    cov_final.to_excel(writer, sheet_name='Covariances')
    pd.Series(vol_final, index=returns.columns, name='Volatility').to_frame().to_excel(writer, sheet_name='Volatilities')
    corr_final.to_excel(writer, sheet_name='Correlations')